In [ ]:
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Environment, Model
from azureml.core.compute import ComputeTarget

In [ ]:
from azureml.core.runconfig import RunConfiguration, CondaDependencies, DEFAULT_CPU_IMAGE
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PublishedPipeline
from azureml.pipeline.core import StepSequence
from azureml.widgets import RunDetails

In [ ]:
import pandas as pd
import numpy as np
import os
import random as r

In [ ]:
ws = Workspace.from_config()

In [ ]:
compute_name = 'compute-cluster'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
os.makedirs('Scoring_Scripts', exist_ok=True)  

In [ ]:
%%writefile Scoring_Scripts/Iris_Scoring_Local.py
from azureml.core import Run, Workspace
from azureml.core import Dataset, Datastore, Model

import os
import joblib
import numpy as np
import pandas as pd

run = Run.get_context()

def main():
    ws = run.experiment.workspace
    datastore = Datastore.get_default(ws)
    model_path = Model.get_model_path('Iris-Multi-Classification-AutoML')
    model = joblib.load(model_path)
    dataset = Dataset.get_by_name(ws,'Iris Scoring')
    scoringDF = dataset.to_pandas_dataframe()
    predictions = model.predict(scoringDF)
    predSeries = pd.Series(predictions)
    scoringDF['Prediction'] = predSeries

    output_datastore_path = 'Output_Folder'
    os.makedirs(output_datastore_path, exist_ok=True) 
    FileName = "Iris_Predictions.csv"
    OutputPath = os.path.join(output_datastore_path, FileName)
    scoringDF.to_csv(OutputPath, index = False, sep=',')
    datastore.upload_files(files=[OutputPath], target_path=output_datastore_path, overwrite=True)
    os.remove(OutputPath)
    os.rmdir(output_datastore_path)
    
if __name__ == '__main__':
    main()

In [ ]:
# Create an Environment (collection of Python packages) you will use to run the script
Env = Environment(name='AutoML Environment')

# Create a CondaDepencies variable
conda_dep = CondaDependencies()

# Add conda packages
conda_dep.add_conda_package("numpy==1.18.5")
conda_dep.add_conda_package("joblib==0.14.1")
conda_dep.add_conda_package("pandas==0.25.3")
conda_dep.add_conda_package("packaging==20.7")
conda_dep.add_conda_package("xgboost==0.90")

# Add pip packages
conda_dep.add_pip_package("azureml-defaults==1.19.0")
conda_dep.add_pip_package("azureml-automl-core==1.19.0")
conda_dep.add_pip_package("azureml-automl-runtime==1.19.0")

# Adds dependencies to PythonSection of myenv
Env.python.conda_dependencies=conda_dep

# Register the environment to your workspace
RegisteredEnvironment = Env.register(workspace=ws)

In [ ]:
run_config = RunConfiguration()
run_config.environment = Env
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

In [ ]:
# Create the training (Estimator) step by filling in all of the parameters
scoring_step = PythonScriptStep(name='iris-scoring-step',
                                         script_name='Iris_Scoring_Local.py',
                                         source_directory='Scoring_Scripts',
                                         arguments=[],
                                         inputs=[], 
                                         compute_target=compute_target,
                                         runconfig=run_config,
                                         allow_reuse=False)

In [ ]:
step_sequence = StepSequence(steps=[scoring_step])
pipeline = Pipeline(workspace=ws, steps=step_sequence)

In [ ]:
# Run your pipeline
pipeline_experiment = Experiment(ws, 'Iris-Scoring-Pipeline-Run')
pipeline_run = pipeline_experiment.submit(pipeline, show_output=True)

In [ ]:
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name='Iris-Local-Scoring-Pipeline',\
    description='Pipeline that Scores Iris Data', version= '1.0')

published_pipeline